In [12]:
import os
import librosa
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.models import load_model

In [13]:
#load the dataset 
data_dir = 'C:\\Users\\hp\\Desktop\\infant baby cry\\donateacry_corpus'
classes = ['belly_pain','burping','discomfort','hungry','tired'] 

In [14]:
def load_and_preprocess_data(data_dir, classes, target_shape=(128, 128)):
    data = []
    labels = []
    
    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                audio_data, sample_rate = librosa.load(file_path, sr=None)
                mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
                mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
                data.append(mel_spectrogram)
                labels.append(i)
    
    return np.array(data), np.array(labels)


In [15]:
data, labels = load_and_preprocess_data(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))  # Convert labels to one-hot encoding
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [16]:
input_shape = X_train[0].shape
input_layer = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(len(classes), activation='softmax')(x)
model = Model(input_layer, output_layer)

In [17]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 480ms/step - accuracy: 0.5925 - loss: 53.3112 - val_accuracy: 0.6522 - val_loss: 7.0338
Epoch 2/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 465ms/step - accuracy: 0.8587 - loss: 0.8482 - val_accuracy: 0.7391 - val_loss: 2.5768
Epoch 3/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 496ms/step - accuracy: 0.8756 - loss: 0.6239 - val_accuracy: 0.7500 - val_loss: 1.9788
Epoch 4/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 507ms/step - accuracy: 0.8966 - loss: 0.4890 - val_accuracy: 0.7609 - val_loss: 2.0028
Epoch 5/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 442ms/step - accuracy: 0.8993 - loss: 0.4727 - val_accuracy: 0.7609 - val_loss: 2.1865
Epoch 6/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 495ms/step - accuracy: 0.9130 - loss: 0.3656 - val_accuracy: 0.7609 - val_loss: 2.4325
Epoch 7/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 451ms/step - accuracy: 0.9438 - loss: 0.2541 - val_accuracy: 0.7717 - val_loss: 2.6830
Epoch 8/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 475ms/step - accuracy: 0.9219 - loss: 0.2277 - val_accura